# Toronto's Neighbourhoods

**This notebook will gather data on, perform segmentation and clustering of Toronto's neighbourhoods**

Let us first import the required packages and libraries

In [3]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

### Getting the neighbourhood Dataframe

Next, we will get the data from the following Wikipedia page- https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [4]:

path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # Naming the link urla


toronto_data=pd.read_html(path,match='Borough',na_values='Not assigned')[0]#Reading data from url page



In [5]:
toronto_data.head()


,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront




Now that the raw dataset has been obtained, we will further refine it

In [6]:
df=toronto_data.dropna(axis=0,thresh=2) #Removing the NaN values
df=df.reset_index(drop=True).fillna(value="Queen's Park")# Resetting the index and filling the NaN value
df.head()#Show results 

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


Now, let us group together neighbourhoods sharing the same postcode

In [7]:
df1=df.drop(columns=['Neighbourhood']) # Create new dataframe by dropping the 'Neighbourhood' column
df1.drop_duplicates(subset="Postcode",inplace=True) # Remove duplicate rows
df1 = df1.groupby('Postcode')['Borough'].apply(list).reset_index() #Groups together the boroughs with same postcode 

In [8]:

df2 = df.groupby('Postcode')['Neighbourhood'].apply(list).reset_index() # New dataframe which groups together the neighbourhoods with same Postcode
df2=df2.drop(columns=['Postcode'])# Dropping the column 'Postcode'
                                        

In [9]:
df1.head()

,Postcode,Borough
0,M1B,[Scarborough]
1,M1C,[Scarborough]
2,M1E,[Scarborough]
3,M1G,[Scarborough]
4,M1H,[Scarborough]


In [10]:
df2.head()

,Neighbourhood
0,"[Rouge, Malvern]"
1,"[Highland Creek, Rouge Hill, Port Union]"
2,"[Guildwood, Morningside, West Hill]"
3,[Woburn]
4,[Cedarbrae]


In [11]:
df=pd.concat([df1,df2],axis=1) #Adding together the two Dataframes

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,[Scarborough],"[Rouge, Malvern]"
1,M1C,[Scarborough],"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,[Scarborough],"[Guildwood, Morningside, West Hill]"
3,M1G,[Scarborough],[Woburn]
4,M1H,[Scarborough],[Cedarbrae]


In [13]:
df['Borough']=df['Borough'].str.get(0)# Removes the square brackets from 'Boroughs' column

In [14]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [15]:
i=0
while(i<df.shape[0]-1):
    df['Neighbourhood'][i]=",".join(map(str,df['Neighbourhood'][i]))  # Removes the square brackets from 'Neighbourhood' column
    i=i+1
    



In [16]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
print('Number of rows in Toronto Neighbourhoods dataframe is {} '.format(df.shape[0])) # Prints the number of rows

Number of rows in Toronto Neighbourhoods dataframe is 103 


### Getting the geographical co-ordinates

Now, let us get the coordinates for the respective postcodes

In [18]:
 # Downloading the coordinates data from the url

!wget -q -O 'Coordinate_data.csv' http://cocl.us/Geospatial_data

In [19]:
latlng_data=pd.read_csv('Coordinate_data.csv') # Converting into dataframe

latlng_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df = pd.merge(df, latlng_data, left_on='Postcode', right_on='Postal Code') # Merging together the two dataframes

In [21]:
df=df.drop(columns=['Postal Code']) # Removing the duplicate column

In [22]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Segmentation and clustering 

Now that we have all the required data, we can further explore the neighbourhoods

First, let's generate a map of Toronto with all of our postcodes labelled

In [23]:
# First, we need to get the geographical coordinates of Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postcode, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Postcode'], df['Neighbourhood']):
    label = '{}-{}'.format(neighbourhood, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's explore the neighbourhood of Downtown Toronto

In [25]:
downtoronto_df=df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True) 
downtoronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [26]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Downtown Toronto are 43.654027, -79.3802003.


In [27]:
map_downtoronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtoronto_df['Latitude'], downtoronto_df['Longitude'], downtoronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtoronto)  
    
map_downtoronto

Now, let us use the Foursquare API to explore this area

In [28]:
CLIENT_ID = 'YG2GD4DIBG0I5ITAWT2GTED5KK2SPZ53IRLCJ1RMWNS2E3YU' # your Foursquare ID
CLIENT_SECRET = 'CKVWEOJ1JXRIR1MA0K3C53JNE0QI2Z0X01ENZ4OAEN5ZVAYA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: YG2GD4DIBG0I5ITAWT2GTED5KK2SPZ53IRLCJ1RMWNS2E3YU
CLIENT_SECRET:CKVWEOJ1JXRIR1MA0K3C53JNE0QI2Z0X01ENZ4OAEN5ZVAYA


Let's look at a particular area within Downtown Toronto

In [29]:
downtoronto_df.loc[0,'Neighbourhood']

'Rosedale'

In [30]:
neighbourhood_latitude = downtoronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = downtoronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = downtoronto_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


#### Let's get the top 50 venues in a 1 km radius

In [31]:
LIMIT=50
radius=1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [32]:
results = requests.get(url).json() # To obtain the results of the search
results

{'meta': {'code': 200, 'requestId': '5dd6a78db9a389001bedcd26'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.68856260900001,
    'lng': -79.36510816548741},
   'sw': {'lat': 43.670562590999985, 'lng': -79.38995063451262}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcb343f964a520e32e21e3',
       'name': 'Summerhill Market',
       'location': {'address': '446 Summerhill Ave',
        'crossStreet': 'btwn. MacLennan Ave. and Glen Rd.',
        'lat': 43.68626482142425,
        'lng': -79.37545823237794,
      

In [33]:
def get_category_type(row):                          # Function to get the category of the venue
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Black Camel,BBQ Joint,43.677016,-79.389367
2,Toronto Lawn Tennis Club,Athletics & Sports,43.680667,-79.388559
3,Maison Selby,Bistro,43.671232,-79.376618
4,Tinuno,Filipino Restaurant,43.671281,-79.374920


In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


#### Let's replicate this for all neighbourhoods

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000): # Function to explore all neighbourhoods
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)       
    
    
    

Now,for running the above function on each neighbourhood in Downtown Toronto, we proceed as follows

In [53]:
Downtoronto_venues = getNearbyVenues(names=downtoronto_df['Neighbourhood'],
                                   latitudes=downtoronto_df['Latitude'],
                                   longitudes=downtoronto_df['Longitude']
                                  )


Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [54]:
print(Downtoronto_venues.shape) # The resulting dataframe
Downtoronto_venues.head()

(825, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,43.679563,-79.377529,Black Camel,43.677016,-79.389367,BBQ Joint
2,Rosedale,43.679563,-79.377529,Toronto Lawn Tennis Club,43.680667,-79.388559,Athletics & Sports
3,Rosedale,43.679563,-79.377529,Maison Selby,43.671232,-79.376618,Bistro
4,Rosedale,43.679563,-79.377529,Tinuno,43.671281,-79.374920,Filipino Restaurant


To check number of venues for each neighbourhood:

In [55]:
Downtoronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",37,37,37,37,37,37
Central Bay Street,50,50,50,50,50,50
"Chinatown,Grange Park,Kensington Market",50,50,50,50,50,50
Christie,50,50,50,50,50,50
Church and Wellesley,50,50,50,50,50,50
"Commerce Court,Victoria Hotel",50,50,50,50,50,50


To find number of unique categories

In [56]:
print('There are {} uniques categories.'.format(len(Downtoronto_venues['Venue Category'].unique())))

There are 173 uniques categories.


#### Analyzing each neighbourhood

In [58]:
# one hot encoding
downtoronto_onehot = pd.get_dummies(Downtoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtoronto_onehot['Neighbourhood'] = Downtoronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtoronto_onehot.columns[-1]] + list(downtoronto_onehot.columns[:-1])
downtoronto_onehot = downtoronto_onehot[fixed_columns]

downtoronto_onehot.head()

,Neighbourhood,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Opera House,Organic Grocery,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [59]:
downtoronto_onehot.shape

(825, 174)

Grouping by neighbourhood and taking the mean of frequency of each category

In [60]:
downtoronto_grouped = downtoronto_onehot.groupby('Neighbourhood').mean().reset_index()
downtoronto_grouped 

,Neighbourhood,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Opera House,Organic Grocery,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Adelaide,King,Richmond",0.0000,0.0000,0.060000,0.00,0.00,0.00,0.00,0.04,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.02,0.02,0.020000,0.00,0.00,0.02,0.02,0.080000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.060000,0.00,0.00,0.00,0.00,0.04,0.000000,0.02,0.00,0.00,0.000000,0.000000,0.02,0.00,0.00,0.000000,0.0000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.00,0.02,0.000000,0.02,0.02,0.000,0.00,0.00,0.00,0.00,0.000000,0.04,0.00,0.000000,0.00,0.000000,0.000000,0.02,0.000000,0.02,0.0,0.00,0.00,0.00,0.02,0.02,0.00,0.000000,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.02,0.000000,0.02,0.00,0.000000,0.00,0.000000,0.00,0.02,0.020000,0.000000,0.00,0.00,0.02,0.02,0.000000,0.0000,0.00,0.0000,0.02,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.02,0.00,0.00,0.040000,0.00,0.02,0.00,0.00,0.000000,0.00,0.00,0.00,0.020000,0.020000,0.00,0.0000,0.000000,0.00,0.0000,0.02,0.00,0.00,0.00,0.00
1,Berczy Park,0.0000,0.0000,0.000000,0.00,0.00,0.02,0.00,0.00,0.000000,0.000000,0.00,0.04,0.000000,0.00,0.00,0.02,0.02,0.06,0.00,0.020000,0.00,0.00,0.020000,0.00,0.00,0.00,0.00,0.060000,0.000000,0.000000,0.02,0.00,0.00,0.00,0.02,0.04,0.060000,0.00,0.00,0.00,0.00,0.02,0.000000,0.02,0.02,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0000,0.00,0.00,0.00,0.00,0.00,0.000000,0.04,0.00,0.000000,0.00,0.00,0.02,0.02,0.02,0.00,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.020000,0.02,0.00,0.000,0.00,0.00,0.00,0.00,0.000000,0.06,0.00,0.000000,0.00,0.000000,0

In [62]:
downtoronto_grouped.shape # New size

(18, 174)

#### Now, printing each neighbourhood with the top 5 most common venues

In [63]:
num_top_venues = 5

for hood in downtoronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtoronto_grouped[downtoronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0                 Café  0.08
1  American Restaurant  0.06
2          Coffee Shop  0.06
3                Hotel  0.04
4     Asian Restaurant  0.04


----Berczy Park----
            venue  freq
0           Hotel  0.06
1            Café  0.06
2        Beer Bar  0.06
3     Coffee Shop  0.06
4  Farmers Market  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
             venue  freq
0  Harbor / Marina  0.12
1      Coffee Shop  0.12
2             Café  0.12
3            Track  0.06
4           Garden  0.06


----Cabbagetown,St. James Town----
                 venue  freq
0                 Park  0.08
1                 Café  0.05
2                Diner  0.05
3           Restaurant  0.05
4  Japanese Restaurant  0.05


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.14
1   Ice Cream Shop  0.04
2         Tea Room  0.04
3             Café  0.04
4  

In [64]:
def return_most_common_venues(row, num_top_venues): # Function to sort venues in descending order
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = downtoronto_grouped['Neighbourhood']

for ind in np.arange(downtoronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtoronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,American Restaurant,Coffee Shop,Steakhouse,Hotel,Asian Restaurant,Concert Hall,Seafood Restaurant,Pizza Place,Smoke Shop
1,Berczy Park,Beer Bar,Coffee Shop,Hotel,Café,Bakery,Park,Seafood Restaurant,Cocktail Bar,Farmers Market,French Restaurant
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Harbor / Marina,Café,Airport,Sculpture Garden,Dog Run,Scenic Lookout,Park,Track,Tunnel
3,"Cabbagetown,St. James Town",Park,Café,Diner,Restaurant,Gastropub,Coffee Shop,Japanese Restaurant,Filipino Restaurant,Italian Restaurant,Rock Club
4,Central Bay Street,Coffee Shop,Ice Cream Shop,Bubble Tea Shop,Italian Restaurant,Tea Room,Café,Chinese Restaurant,Japanese Restaurant,Plaza,Poke Place


#### Now, we cluster the neighbourhood by k-means clustering

In [67]:
# set number of clusters
kclusters = 5

downtoronto_grouped_clustering = downtoronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 1, 0, 4, 0, 0, 4, 3, 3], dtype=int32)

Creating a new dataframe which includes the cluster as well as top 10 venues per neighbourhood

In [68]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtoronto_merged = downtoronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtoronto_merged = downtoronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtoronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Coffee Shop,Grocery Store,Convenience Store,Bank,Pie Shop,Sandwich Place,Candy Store,Filipino Restaurant,Smoothie Shop
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Park,Café,Diner,Restaurant,Gastropub,Coffee Shop,Japanese Restaurant,Filipino Restaurant,Italian Restaurant,Rock Club
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,4,Coffee Shop,Men's Store,Japanese Restaurant,Dance Studio,Thai Restaurant,Gastropub,Restaurant,Gay Bar,Park,Mexican Restaurant
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,4,Coffee Shop,Park,Pub,Bakery,Italian Restaurant,Café,Performing Arts Venue,Theater,Mexican Restaurant,Breakfast Spot
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,4,Clothing Store,Coffee Shop,Cosmetics Shop,Restaurant,Ramen Restaurant,Theater,Italian Restaurant,American Restaurant,Miscellaneous Shop,Burrito Place


Creating a map to visualize all the clusters

In [69]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtoronto_merged['Latitude'], downtoronto_merged['Longitude'], downtoronto_merged['Neighbourhood'], downtoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [70]:
downtoronto_merged.loc[downtoronto_merged['Cluster Labels'] == 0, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Park,Café,Diner,Restaurant,Gastropub,Coffee Shop,Japanese Restaurant,Filipino Restaurant,Italian Restaurant,Rock Club
5,Downtown Toronto,0,Coffee Shop,Café,Gastropub,Italian Restaurant,Farmers Market,Bakery,Hotel,Japanese Restaurant,Beer Bar,Restaurant
6,Downtown Toronto,0,Beer Bar,Coffee Shop,Hotel,Café,Bakery,Park,Seafood Restaurant,Cocktail Bar,Farmers Market,French Restaurant
9,Downtown Toronto,0,Hotel,Coffee Shop,Plaza,Baseball Stadium,Café,Aquarium,Park,Brewery,Scenic Lookout,Salad Place
12,Downtown Toronto,0,Bakery,Café,Bookstore,Pizza Place,Beer Bar,Japanese Restaurant,Vegetarian / Vegan Restaurant,Park,Greek Restaurant,Concert Hall
13,Downtown Toronto,0,Café,Mexican Restaurant,Comfort Food Restaurant,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Caribbean Restaurant,Bakery,Burger Joint
15,Downtown Toronto,0,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Beer Bar,Bakery,Coffee Shop,Seafood Restaurant,Farmers Market,Cocktail Bar
17,Downtown Toronto,0,Korean Restaurant,Café,Coffee Shop,Cocktail Bar,Grocery Store,Comedy Club,Indian Restaurant,Pizza Place,South American Restaurant,Latin American Restaurant


### Cluster 2

In [71]:
downtoronto_merged.loc[downtoronto_merged['Cluster Labels'] == 1, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,Coffee Shop,Harbor / Marina,Café,Airport,Sculpture Garden,Dog Run,Scenic Lookout,Park,Track,Tunnel


### Cluster 3

In [72]:
downtoronto_merged.loc[downtoronto_merged['Cluster Labels'] == 2, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Park,Coffee Shop,Grocery Store,Convenience Store,Bank,Pie Shop,Sandwich Place,Candy Store,Filipino Restaurant,Smoothie Shop


### Cluster 4

In [73]:
downtoronto_merged.loc[downtoronto_merged['Cluster Labels'] == 3, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,3,Café,American Restaurant,Coffee Shop,Steakhouse,Hotel,Asian Restaurant,Concert Hall,Seafood Restaurant,Pizza Place,Smoke Shop
10,Downtown Toronto,3,Café,Coffee Shop,Hotel,Salad Place,American Restaurant,Restaurant,Concert Hall,Steakhouse,Gastropub,Deli / Bodega
11,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Steakhouse,American Restaurant,Deli / Bodega,Hotel,Japanese Restaurant,Beer Bar,Gastropub
16,Downtown Toronto,3,Café,Coffee Shop,American Restaurant,Gym,Asian Restaurant,Steakhouse,Restaurant,Concert Hall,Deli / Bodega,Hotel


### Cluster 5

In [75]:
downtoronto_merged.loc[downtoronto_merged['Cluster Labels'] == 4, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Coffee Shop,Men's Store,Japanese Restaurant,Dance Studio,Thai Restaurant,Gastropub,Restaurant,Gay Bar,Park,Mexican Restaurant
3,Downtown Toronto,4,Coffee Shop,Park,Pub,Bakery,Italian Restaurant,Café,Performing Arts Venue,Theater,Mexican Restaurant,Breakfast Spot
4,Downtown Toronto,4,Clothing Store,Coffee Shop,Cosmetics Shop,Restaurant,Ramen Restaurant,Theater,Italian Restaurant,American Restaurant,Miscellaneous Shop,Burrito Place
7,Downtown Toronto,4,Coffee Shop,Ice Cream Shop,Bubble Tea Shop,Italian Restaurant,Tea Room,Café,Chinese Restaurant,Japanese Restaurant,Plaza,Poke Place
